# 결정 트리 실습

사용자 행동 인식 데이터 세트

[원본 데이터셋]
https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones

## 종합 코드에 하이퍼파라미터를 제외한 코드를 모아놨습니다.

데이터 세트 정보:

실험은 19-48세 사이의 30명의 지원자 그룹으로 수행되었습니다. 각자 스마트폰(Samsung Galaxy S II)을 허리에 차고 6가지 활동(
1:WALKING, 
2:WALKING_UPSTAIRS, 
3:WALKING_DOWNSTAIRS, 4:SITTING, 5:STANDING, 6:LAYING)을 수행했습니다. 내장된 가속도계와 자이로스코프를 사용하여 50Hz의 일정한 속도로 3축 선형 가속도와 3축 각속도를 캡처했습니다. 실험은 데이터에 수동으로 레이블을 지정하기 위해 비디오로 녹화되었습니다. 얻은 데이터 세트는 훈련 데이터 생성을 위해 70%, 테스트 데이터 생성을 위해 30%가 선택된 두 세트로 무작위로 분할되었습니다.

센서 신호(가속도계 및 자이로스코프)는 노이즈 필터를 적용하여 사전 처리된 다음 2.56초 및 50% 중첩(128 판독/창)의 고정 너비 슬라이딩 창에서 샘플링되었습니다. 중력 및 신체 운동 성분을 갖는 센서 가속도 신호는 Butterworth 저역 통과 필터를 사용하여 신체 가속도와 중력으로 분리되었습니다. 중력은 저주파 성분만 있다고 가정하므로 차단 주파수가 0.3Hz인 필터를 사용했습니다. 각 창에서 시간 및 주파수 영역에서 변수를 계산하여 특징 벡터를 얻었습니다.

이 데이터 세트에 대한 자세한 내용은 README.txt 파일을 확인하십시오.

참가자 중 한 명과 함께 녹화된 6가지 활동의 예를 포함하는 실험 비디오는 다음 링크에서 볼 수 있습니다.

https://www.youtube.com/watch?v=XOEN9W05_4A]


**Mission**

수집된 데이터를 기반으로 결정트리를 이용해 어떤 행동을 하고 있는지 예측해 봅시다.

In [ ]:
# ## 코랩을 사용할 때
# #드라이브 마운트
# from google.colab import drive
# drive.mount('/content/drive')

# #현재 작업 위치 이동
# #띄어쓰기에 \붙일 것
# %cd /content/drive/Othercomputers/내\ 노트북_before/Devpy/13.머신러닝

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## 데이터 로드

1. 피처만 있는 파일을 로드
2. 훈련데이터와 피처를 합쳐 훈련 데이터 프레임 만듬
3. 데스트데이터와 피처를 합쳐 테스트 데이터 프레임 만듬

In [96]:
# 피처이름 로드 
feature_name_df = pd.read_csv('./data/human_activity/features_new.txt', header=None,names=['column_name'])
feature_name_df.head()


,column_name
0,tBodyAcc-mean()-X
1,tBodyAcc-mean()-Y
2,tBodyAcc-mean()-Z
3,tBodyAcc-std()-X
4,tBodyAcc-std()-Y


In [97]:
# 피처이름을 리스트로
feature_name = feature_name_df['column_name'].values.tolist()
len(feature_name)

561

In [102]:
# 학습 피처 데이터 셋과 테스트 피처 데이터을 DataFrame으로 로딩. 컬럼명은 feature_name 적용
h_acc_train=pd.read_csv('./data/human_activity/train/X_train.txt',header=None)
# h_acc_train
h_acc_split=h_acc_train[0].str.split(' ')
h_acc_list=[]
for n in h_acc_split:
    n=list(filter(len,n))
    h_acc_list.append(n)

h_acc_np=np.array(h_acc_list)
# h_acc_df

In [111]:
h_acc_test=pd.read_csv('./data/human_activity/test/X_test.txt',header=None)
# h_acc_train
h_acc_split1=h_acc_test[0].str.split(' ')
h_acc_list2=[]
for n in h_acc_split1:
    n=list(filter(len,n))
    h_acc_list2.append(n)

h_acc_np2=np.array(h_acc_list2)

In [ ]:
def read_file(filepath):
    h_acc_test=pd.read_csv(filepath,header=None)
    h_acc_split=h_acc_test[0].str.split(' ')
    h_acc_list=[]
    for n in h_acc_split:
        n=list(filter(len,n))
        h_acc_list.append(n)
    h_acc_np=np.array(h_acc_list)

    return h_acc_np


In [126]:
X_train=pd.DataFrame(data=h_acc_np, columns=feature_name)
X_train.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,2.8858451e-001,-2.0294171e-002,-1.3290514e-001,-9.9527860e-001,-9.8311061e-001,-9.1352645e-001,-9.9511208e-001,-9.8318457e-001,-9.2352702e-001,-9.3472378e-001,...,-7.4323027e-002,-2.9867637e-001,-7.1030407e-001,-1.1275434e-001,3.0400372e-002,-4.6476139e-001,-1.8445884e-002,-8.4124676e-001,1.7994061e-001,-5.8626924e-002
1,2.7841883e-001,-1.6410568e-002,-1.2352019e-001,-9.9824528e-001,-9.7530022e-001,-9.6032199e-001,-9.9880719e-001,-9.7491437e-001,-9.5768622e-001,-9.4306751e-001,...,1.5807454e-001,-5.9505094e-001,-8.6149931e-001,5.3476955e-002,-7.4345661e-003,-7.3262621e-001,7.0351059e-001,-8.4478760e-001,1.8028889e-001,-5.4316717e-002
2,2.7965306e-001,-1.9467156e-002,-1.1346169e-001,-9.9537956e-001,-9.6718701e-001,-9.7894396e-001,-9.9651994e-001,-9.6366837e-001,-9.7746859e-001,-9.3869155e-001,...,4.1450281e-001,-3.9074815e-001,-7.6010372e-001,-1.1855926e-001,1.7789948e-001,1.0069921e-001,8.0852908e-001,-8.4893347e-001,1.8063731e-001,-4.9117815e-002
3,2.7917394e-001,-2.6200646e-002,-1.2328257e-001,-9.9609149e-001,-9.8340270e-001,-9.9067510e-001,-9.9709947e-001,-9.8274984e-001,-9.8930250e-001,-9.3869155e-001,...,4.0457253e-001,-1.1729020e-001,-4.8284451e-001,-3.6787973e-002,-1.2892494e-002,6.4001104e-001,-4.8536645e-001,-8.4864938e-001,1.8193476e-001,-4.7663183e-002
4,2.7662877e-001,-1.6569655e-002,-1.1536185e-001,-9.9813862e-001,-9.8081727e-001,-9.9048163e-001,-9.9832113e-001,-9.7967187e-001,-9.9044113e-001,-9.4246912e-001,...,8.7753013e-002,-3.5147093e-001,-6.9920515e-001,1.2332005e-001,1.2254196e-001,6.9357829e-001,-6.1597061e-001,-8.4786525e-001,1.8515116e-001,-4.3892254e-002


In [127]:
X_test=pd.DataFrame(data=h_acc_np2, columns=feature_name)
X_test.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,2.5717778e-001,-2.3285230e-002,-1.4653762e-002,-9.3840400e-001,-9.2009078e-001,-6.6768331e-001,-9.5250112e-001,-9.2524867e-001,-6.7430222e-001,-8.9408755e-001,...,7.1645446e-002,-3.3037044e-001,-7.0597388e-001,6.4624029e-003,1.6291982e-001,-8.2588562e-001,2.7115145e-001,-7.2000927e-001,2.7680104e-001,-5.7978304e-002
1,2.8602671e-001,-1.3163359e-002,-1.1908252e-001,-9.7541469e-001,-9.6745790e-001,-9.4495817e-001,-9.8679880e-001,-9.6840133e-001,-9.4582340e-001,-8.9408755e-001,...,-4.0118872e-001,-1.2184509e-001,-5.9494387e-001,-8.3494968e-002,1.7499572e-002,-4.3437455e-001,9.2059323e-001,-6.9809082e-001,2.8134292e-001,-8.3898014e-002
2,2.7548482e-001,-2.6050420e-002,-1.1815167e-001,-9.9381904e-001,-9.6992551e-001,-9.6274798e-001,-9.9440345e-001,-9.7073498e-001,-9.6348267e-001,-9.3926027e-001,...,6.2891313e-002,-1.9042189e-001,-6.4073573e-001,-3.4956250e-002,2.0230203e-001,6.4103354e-002,1.4506843e-001,-7.0277146e-001,2.8008303e-001,-7.9346197e-002
3,2.7029822e-001,-3.2613869e-002,-1.1752018e-001,-9.9474279e-001,-9.7326761e-001,-9.6709068e-001,-9.9527433e-001,-9.7447101e-001,-9.6889736e-001,-9.3860975e-001,...,1.1669529e-001,-3.4441804e-001,-7.3612380e-001,-1.7067021e-002,1.5443783e-001,3.4013408e-001,2.9640709e-001,-6.9895383e-001,2.8411379e-001,-7.7108002e-002
4,2.7483295e-001,-2.7847788e-002,-1.2952716e-001,-9.9385248e-001,-9.6744548e-001,-9.7829499e-001,-9.9411140e-001,-9.6595259e-001,-9.7734600e-001,-9.3860975e-001,...,-1.2171128e-001,-5.3468487e-001,-8.4659517e-001,-2.2226521e-003,-4.0046393e-002,7.3671509e-001,-1.1854473e-001,-6.9224496e-001,2.9072202e-001,-7.3856810e-002


In [128]:
label_train=pd.read_csv('./data/human_activity/train/y_train.txt',header=None,names=['Action'])
label_train.head()

,Action
0,5
1,5
2,5
3,5
4,5


In [129]:
label_test=pd.read_csv('./data/human_activity/test/y_test.txt',header=None,names=['Action'])
label_test.head()

,Action
0,5
1,5
2,5
3,5
4,5


In [ ]:
# 학습 레이블과 테스트 레이블 데이터을 DataFrame으로 로딩하고 컬럼명은 action으로 부여

In [130]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
# DesisionTree분류 클래스 사용(randomstate 156)
dt_clf=DecisionTreeClassifier(random_state=156)
# 훈련
#train= action labeltrain 
#test ac_test labeltest
# X_train,X_test,y_train,y_test=train_test_split(action.values,label_train.values,test_size=0.3,random_state=156)

dt_clf.fit(X_train, label_train)
# dt_clf.fit(X_train, y_train)

# 예측
pred=dt_clf.predict(X_test)
# pred=dt_clf.predict(X_test)

#평가
# accuracy_score(y_test,pred)
accuracy_score(label_test,pred)


0.8547675602307431

In [125]:
# DecisionTreeClassifier의 하이퍼 파라미터 추출
print('DecisionTreeClassifier 기본 하이퍼 파라미터:\n', dt_clf.get_params())

DecisionTreeClassifier 기본 하이퍼 파라미터:
 {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 156, 'splitter': 'best'}


## 하이퍼파라미터 튜닝1

In [143]:
from sklearn.model_selection import GridSearchCV

#max_depth를 바꿔가며 훈련해보자
#나무의 깊이를 얼마만큼 성장시킬것인가

params = {
    'max_depth' : [ 6, 8 ,10, 12, 16 ,20, 24]
}
grid_clf=GridSearchCV(dt_clf,param_grid=params,scoring='accuracy',cv=3)



In [144]:
# GridSearchCV 사용

# 훈련

grid_clf.fit(X_train,label_train)


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [6, 8, 10, 12, 16, 20, 24]},
             scoring='accuracy')

In [146]:
print(f'베스트 파라미터 : {grid_clf.best_params_} 최적 평균 정확도 : {grid_clf.best_score_}')

베스트 파라미터 : {'max_depth': 6} 최적 평균 정확도 : 0.8472506848516641


In [147]:
# GridSearchCV객체의 cv_results_ 속성을 DataFrame으로 생성. 
scores_df=pd.DataFrame(grid_clf.cv_results_)
scores_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,1.767610,0.025864,0.257725,0.000472,6,{'max_depth': 6},0.839249,0.868217,0.834286,0.847251,0.014963,1
1,2.082052,0.054012,0.258068,0.003726,8,{'max_depth': 8},0.829457,0.843329,0.839184,0.837323,0.005814,2
2,2.331055,0.117031,0.260024,0.004585,10,{'max_depth': 10},0.831905,0.850265,0.808163,0.830111,0.017235,3
3,2.484993,0.088176,0.257382,0.002404,12,{'max_depth': 12},0.838433,0.835985,0.808571,0.827663,0.013537,6
4,2.682045,0.143505,0.255103,0.000409,16,{'max_depth': 16},0.822521,0.838841,0.814694,0.825352,0.010059,7


In [151]:
#베스트 분류기 선택
#예측
#평가

estimator=grid_clf.best_estimator_
pred=estimator.predict(X_test)
# pred
accuracy_score(label_test,pred)


0.8557855446216491

```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

## 하이퍼파라미터 튜닝2

In [152]:
#하이퍼파라미터
params = {
    'max_depth' : [ 8 , 12, 16 ,20], 
    'min_samples_split' : [16,24],
}


In [153]:
# GridSearchCV 사용
grid_clf2=GridSearchCV(dt_clf,param_grid=params,scoring='accuracy',cv=3)

# 훈련
grid_clf2.fit(X_train,label_train)


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [8, 12, 16, 20],
                         'min_samples_split': [16, 24]},
             scoring='accuracy')

In [157]:
#베스트 분류기 선택
# print(f'베스트파라미텨{grid_clf2.best_params_} 최적의 평균정확도{grid_clf2.best_score_}')
#max depth : 8 min_samples_split=16
scores_df2=pd.DataFrame(grid_clf2.cv_results_)
scores_df2.head()
#예측
estimator2=grid_clf2.best_estimator_
pred=estimator2.predict(X_test)
#평가
accuracy_score(label_test,pred)

0.8717339667458432

```
    하이퍼파라미터 튜닝으로 성능이 향상 되었나요?
```

---

**[생활탐구]**


```
방문을 꼭 닫은채 공부하고 있다는 보검이...
정말 보검이는 공부하고 있을까요?
생체 신호 측정기를 착용한 보검이로 부터 아래와 같은 신호를 가져왔습니다. 
지금 현재 보검이는 무엇을 하고 있나요?

[보검이 생체 신호 파일]X_quiz.txt 

```

In [166]:
#신호 읽어서 예측해보기
bogums=pd.read_csv('./data/X_quiz55.txt',header=None,names=feature_name)
# bogums_df=pd.DataFrame(data=bogums.values,columns=feature_name)
bogums

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.173769,-0.024869,-0.094733,-0.893862,-0.944204,-0.960824,-0.904192,-0.950321,-0.967834,-0.923737,...,-0.267475,0.199002,-0.324917,-0.089103,0.10531,0.183368,-0.169434,0.751465,-0.484928,-0.505


In [162]:
dt_clf2=DecisionTreeClassifier(random_state=156, min_samples_split=16, max_depth=8)
dt_clf2.fit(X_train,label_train)

DecisionTreeClassifier(max_depth=8, min_samples_split=16, random_state=156)

In [163]:
pred=dt_clf2.predict(bogums)
pred
# 답은 6나온다.

array([6], dtype=int64)

## 종합 코드

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

# X_train = pd.read_csv('./dataset/human_activity/train/X_train.txt',sep='\s+', names=feature_name )
# X_test = pd.read_csv('./dataset/human_activity/test/X_test.txt',sep='\s+', names=feature_name)
# 데이터 로드 하는 과정해서 고민하다가 만들어봤습니다.
def read_file(filepath,column):
    file_df=pd.read_csv(filepath,header=None)
    data_split=file_df[0].str.split(' ')
    data_list=[]
    for n in data_split:
        n=list(filter(len,n))
        data_list.append(n)
    data_np=np.array(data_list)
    data_df=pd.DataFrame(data=data_np, columns=column)
    return data_df


#data load
feature_name_df=pd.read_csv('./data/human_activity/features_new.txt',header=None,names=['columns_name'])
feature_name=feature_name_df['columns_name'].values.tolist()
train_fname='./data/human_activity/train/X_train.txt'
test_fname='./data/human_activity/test/X_test.txt'
X_train=read_file(train_fname,feature_name)
X_test=read_file(test_fname,feature_name)

label_train=pd.read_csv('./data/human_activity/train/y_train.txt',header=None,names=['Action'])
label_train.head()
label_test=pd.read_csv('./data/human_activity/test/y_test.txt',header=None,names=['Action'])
label_test.head()

bogums=pd.read_csv('./data/X_quiz55.txt',header=None,names=feature_name)

#훈련
dt_clf=DecisionTreeClassifier(random_state=156, min_samples_split=16, max_depth=8)
dt_clf.fit(X_train,label_train)
#예측
pred=dt_clf.predict(bogums)
pred

array([6], dtype=int64)